<a href="https://colab.research.google.com/github/Vadiman728/RAG-with-PDFfile/blob/main/RAGwithpdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Для своей работы я построю систему RAG на основе Chat GPT 3.5 Turbo, Поисковик возьму для  википедии. В качестве испытательного кролика будет книга Горького " Тарас Бульба", так как в ней много старорусских выражений (нашел редакцию 1842 года), что будет неким вызовом для модели. Начнем-с

## Установка библиотек

Установим библиотеку для работы с пдф файлами, интегрируем Llama-Hub и HF

In [ ]:
!pip install openai llama_index "arize-phoenix[evals,llama-index]" gcsfs nest-asyncio "openinference-instrumentation-llama-index>=2.0.0" --upgrade

INFO: pip is looking at multiple versions of arize-phoenix[evals,llama-index] to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of arize-phoenix[evals,llama-index] to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of llama-index-llms-openai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of llama-index-program-openai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of llama-parse to determine which version is comp

## Подключаем модель OpenAI

In [ ]:
from llama_index.core.postprocessor import LLMRerank # модуль реранжирования на базе LLM
# Поддержка эмбеддингов и моделей от OpenAI
import openai
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI

Не будем отрываться от идеала- тоже возьмем Chat GPT

In [ ]:
import openai
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

from llama_index.core import (
    VectorStoreIndex,
    GPTVectorStoreIndex,
    SimpleDirectoryReader,
    KeywordTableIndex,
    StorageContext,
    load_index_from_storage,
    ServiceContext,
    Settings,
)

In [ ]:
import getpass # для работы с паролями
import os      # для работы с окружением и файловой системой

# Запрос ввода ключа от OpenAI
os.environ["OPENAI_API_KEY"] = getpass.getpass("Введите OpenAI API Key:")

Введите OpenAI API Key:··········


In [ ]:
# Установим модель по умолчанию

Settings.llm = OpenAI(temperature=0, model='gpt-3.5-turbo')


## Включаем трассировку

In [ ]:
import nest_asyncio
import phoenix as px

from phoenix.evals import (
    HallucinationEvaluator,
    OpenAIModel,
    QAEvaluator,
    RelevanceEvaluator,
    run_evals,
)
from phoenix.session.evaluation import get_qa_with_reference, get_retrieved_documents
from phoenix.trace import DocumentEvaluations, SpanEvaluations

In [ ]:
nest_asyncio.apply()  # необходим для параллельных вычислений в среде ноутбуков

In [ ]:
session = px.launch_app()

🌍 To view the Phoenix app in your browser, visit https://iaczynm5rwj4-496ff2e9c6d22116-6006-colab.googleusercontent.com/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [ ]:
!pip install -U llama_index_core_bridge_pydantic

ERROR: Could not find a version that satisfies the requirement llama_index_core_bridge_pydantic (from versions: none)
ERROR: No matching distribution found for llama_index_core_bridge_pydantic


In [ ]:
from openinference.instrumentation.llama_index import LlamaIndexInstrumentor
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

endpoint = "http://127.0.0.1:6006/v1/traces"
tracer_provider = TracerProvider()
tracer_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter(endpoint)))

LlamaIndexInstrumentor().instrument(tracer_provider=tracer_provider)

ERROR [opentelemetry.instrumentation.instrumentor] DependencyConflict: requested: "llama-index-core >= 0.10.43" but found: "tenacity 9.0.0"


## Импортируем книгу и извлекаем из неё текст

Чтобы книжка быстрее загрузилась (и загрузилась вообще) используем стандартный загрузчик

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Gogol_Taras_Bulba.pdf to Gogol_Taras_Bulba.pdf


После загрузки создаем папку book и закидываем туда нашу книгу, после чего отправляем её на чтение

In [ ]:
# Загружаем файлы PDF
documents = SimpleDirectoryReader('/content/book').load_data()

Воспользуемся таким же индексатором, как в практической

In [ ]:
index = GPTVectorStoreIndex.from_documents(
	documents
)
# Подготавливаем движок к индексу и задем ему вопрос
query_engine = index.as_query_engine()


И позадаем вопросы

In [ ]:
response = query_engine.query("""
Отвечай на вопрос строго по документу, не используй другие источники. Если ответ не обнаружен- пиши "Я не знаю". Вопрос:
Как звали сыновей Тараса Бульбы?
""")
print(response)

Два сына Тараса Бульбы назывались.


Это я первого сентября после летнего чтения 155ти книг...

In [ ]:
response = query_engine.query("""
Отвечай на вопрос строго по документу, не используй другие источники. Если ответ не обнаружен- пиши "Я не знаю". Вопрос:
Кем был Тарас Бульба?
""")
print(response)

Тарас Бульба был козаком.


Хорошо...

In [ ]:
response = query_engine.query("""
Отвечай на вопрос строго по документу, не используй другие источники. Если ответ не обнаружен- пиши "Я не знаю". Вопрос:
Из-за чего Андрий покинул казаков?
""")
print(response)

Андрий покинул казаков из-за просьбы панночки, которая попросила его прийти к ней в город, чтобы либо встретиться с ней, если он ее помнит, либо дать кусок хлеба для старухи, ее матери, если он не помнит ее.


Очень хорошо!

In [ ]:
response = query_engine.query("""
Отвечай на вопрос строго по документу, не используй другие источники. Если ответ не обнаружен- пиши "Я не знаю". Вопрос:
Что в итоге случилось с Остапом и Андрием?
""")
print(response)

Остап присмирился и заснул, а Андрий стоял рядом, испуганный, и когда наконец посмотрел на старого Бульбу, увидел, что тот уже спал, опираясь головой на ладонь.


Последнее по сути правильно, потому как произведение не полное... В оригинале Андрия убил Тарас, а Остап

In [ ]:
print(f"🚀 Открой Phoenix UI для просмотра результата трассировки по ссылке: {session.url}")

🚀 Открой Phoenix UI для просмотра результата трассировки по ссылке: https://iaczynm5rwj5-496ff2e9c6d22116-6006-colab.googleusercontent.com/


## Прикручиваем дополнительный поисковик

В качестве дополнительного поисковика возьмем Википедию. Это универсальный вариант, простой в реализации. Для полноты картины правильнее будет прикрутить какой-нибудь модуль, который по ключевым словам в вопросе подставлял наименования статей в парсер. Для этого поможет модель spacy

In [ ]:
!pip install spacy
!python -m spacy download ru_core_news_sm  # Загрузим модель для русского языка
!pip install llama-index-readers-wikipedia --upgrade
!pip install wikipedia --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 75.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 64.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
  Using cached llama_index_core-0.11.16-py3-none-any.whl.metadata (2.4 kB)
  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 56.5 MB/s eta 0:00:00
  Attempting uninstall: nltk
    Found existing installation: nltk 3.8.1
    Uninstalling nltk-3.8.1:
      Successfully uninstalled nltk

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=ea98b932b61f7cc209951207e71f4f272101cbe597c6fa70fc2b58c45dfc2ece
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [ ]:
import spacy
from llama_index.core import GPTVectorStoreIndex
from llama_index.core import Settings
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.llms.openai import OpenAI
from llama_index.readers.wikipedia import WikipediaReader
from llama_index.embeddings.openai import OpenAIEmbedding

# Загружаем русскую модель spaCy для обработки текста
nlp = spacy.load("ru_core_news_sm")

# Функция для извлечения ключевых слов (существительных и именованных сущностей)
# Чтобы извлекать сразу в именительном падеже используем лемму слов
def extract_keywords(text):
    doc = nlp(text)
    keywords = [token.lemma_ for token in doc if token.pos_ in ['NOUN', 'PROPN']]
    return keywords

# Создаем объект WikipediaReader
wikipedia_reader = WikipediaReader()

# Пример запроса от пользователя
user_query = "Как звали сыновей Тараса Бульбы?"

# Извлекаем ключевые слова из запроса
keywords = extract_keywords(user_query)
print("Ключевые слова:", keywords)

# Загружаем статьи на основе ключевых слов
documents = wikipedia_reader.load_data(pages=keywords)

# Индексация данных для поиска
index = GPTVectorStoreIndex.from_documents(documents)
query_engine = RetrieverQueryEngine.from_args(index.as_retriever())

# Выполняем запрос на основе индекса
response = query_engine.query(user_query)
print(response)

Ключевые слова: ['сын', 'тарас', 'бульба']
Остап и Андрей.


Наконец-то узнали, как их зовут

И еще раз смотрим на трассировку

In [ ]:
print(f"🚀 Открой Phoenix UI для просмотра результата трассировки по ссылке: {session.url}")